# BUILD DATASET FROM HUMAN3.6M [FEEDFORWARD]

## Import

In [1]:
import os
os.chdir("../../")

In [2]:
import sys
import time

import numpy as np
import tensorflow as tf

import math
import random

from src.models.pose3d.linear_model import LinearModel
from src.training import h36m

/home/ubuntu/.conda/envs/hugo_py27/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Load data

In [3]:
data_dir = "dataset/human36M/data/"

actions = ["Directions","Discussion","Eating","Greeting","Phoning","Photo","Posing","Purchases",
           "Sitting","SittingDown","Smoking","Waiting","WalkDog","Walking","WalkTogether"]

number_of_actions = len(actions)

TRAIN_SUBJECTS = [1,5,6,7,8]
TEST_SUBJECTS = [9,11]

trainset_init = h36m.readData( data_dir, TRAIN_SUBJECTS, actions)
testset_init = h36m.readData( data_dir, TEST_SUBJECTS, actions)

trainset_init/testset_init contains the 32 joints provided by h36m the content is loaded from a structure processed
by https://github.com/una-dinosauria/3d-pose-baseline and represent ground truth joint position. 
In h36m module : there is a processRecord function that map the 36 joints to the 13 more relevant ones. 

## Visualize

Human3.6M provides 32 distincts joints, many of them are static.
Only a relevant subset of joints is selected. 

In [6]:
key = next(iter(trainset_init.keys()))
frameId = 0
rotZ = 0.0*math.pi
rotY = 0.0*math.pi
symetry = False
print(key)
print("frame : "+str(frameId) +" / "+str(trainset_init[key].shape[0]))

data = h36m.processRecord(trainset_init[key][frameId], rotZ, rotY, symetry)
print("2d => ")
h36m.visualize_2D(data)
print("3d => ")
h36m.visualize_3D(data)

symetry = True
data = h36m.processRecord(trainset_init[key][frameId], rotZ, rotY, symetry)
print("3d sym => ")
h36m.visualize_3D(data)

rotZ = 0.8*math.pi/2
rotY = -0.1*math.pi/2
data = h36m.processRecord(trainset_init[key][frameId], rotZ, rotY, symetry)
print("3d sym + rotation pi/4 => ")
h36m.visualize_3D(data)



(1, 'Smoking', 'Smoking.h5')
frame : 0 / 2478
2d => 


3d => 


3d sym => 


3d sym + rotation pi/4 => 


## Build the dataset

In [5]:
# jump between frame
def processDataset(initData, jumpingWindow = 10, totalRotation = 30, symetricRatio = 0.7):

    data = []

    for scenes in initData.values():

        for frameId in range(0, scenes.shape[0], jumpingWindow):

            scene = scenes[frameId]
            
            data.append(h36m.processRecord(scene, 0, False))
            if random.random() < symetricRatio:
                data.append(h36m.processRecord(scene, 0, True))
                    
            for i in range(totalRotation):
                rot = random.random()*(2*math.pi)

                data.append(h36m.processRecord(scene, rot, False))

                if random.random() < symetricRatio:
                    data.append(h36m.processRecord(scene, rot, True))
                    
    return np.array(data)
            

In [11]:
trainset = processDataset(trainset_init)

In [12]:
testset = processDataset(testset_init)

In [13]:
np.save("dataset/human36M/processed_data/trainset", trainset)

In [14]:
np.save("dataset/human36M/processed_data/testset", testset)

## Verify Dataset

In [16]:
trainset = np.load("dataset/human36M/processed_data/trainset.npy")

In [16]:
trainset.shape

(2059652, 13, 3)

In [8]:
trainset.shape

(1574837, 13, 3)

In [17]:
h36m.visualize_3D(trainset[3002])

In [17]:
trainset[0,:].shape

(13, 3)

In [6]:
class GaussianNoise:
    
    def __init__(self, std):
        self.std = std
        
    def get(self):
        return np.vectorize(lambda std : np.random.normal(0, std))(self.std)
    
    def getBatch(self, batchSize):
        return np.concatenate([np.expand_dims(self.get(),0) for _ in range(batchSize)],0)
        

In [118]:
"""
Last trained stacked hourglass has the following std dev for input (27/04).
Note : my testset was a bit tricky to predict, therefore i will divide those values by factor 2
0     0.076789
1     0.052730
2     0.187345
3     0.162503
4     0.254326
5     0.222007
6     0.078331
7     0.052955
8     0.188823
9     0.157332
10    0.249365
11    0.218437
12    0.185493
13    0.162686
14    0.153359
15    0.126821
16    0.204631
17    0.166545
18    0.253328
19    0.238201
20    0.152116
21    0.131163
22    0.206065
23    0.166147
24    0.251844
25    0.241080
"""
divideByFactor = 2.0
stdDev = [0.076789,0.052730,0.187345,0.162503,0.254326,0.222007,0.078331,0.052955,0.188823,0.157332,0.249365,0.218437,0.185493,0.162686,0.153359,0.126821,0.204631,0.166545,0.253328,0.238201,0.152116,0.131163,0.206065,0.166147,0.251844,0.241080]
stdDev = np.array(stdDev)
stdDev[:] = 0.025
isXJoints = np.array([i%2 == 0 for i in range(13*2)])
joints_x = stdDev[isXJoints]
joints_y = stdDev[~isXJoints]
curr_res = np.zeros((13,2))
curr_res[:,0] = joints_x.copy()
curr_res[:,1] = joints_y.copy()
stdDev = curr_res 
stdDev = stdDev/divideByFactor
gNoise = GaussianNoise([0.04/2 for _ in range(13*2)])

In [119]:
def viz_(ex):
    a = ex.copy()
    noise = gNoise.get()
    a[:,0] = a[:,0] + noise[:13]
    a[:,2] = a[:,2] + noise[13:]
    h36m.visualize_3D(a)

h36m.visualize_3D(trainset[3002])
viz_(trainset[3002])
viz_(trainset[3002])
viz_(trainset[3002])

In [59]:
gNoise.get()

array([[ 0.01068972,  0.00693554],
       [-0.00903008,  0.0045361 ],
       [-0.00775534, -0.00744019],
       [ 0.02035905,  0.01531951],
       [-0.00924975,  0.01209586],
       [ 0.00542581,  0.00016571],
       [ 0.02161704, -0.02186013],
       [ 0.00011806, -0.0072451 ],
       [-0.00567606,  0.00498873],
       [ 0.00738159,  0.00132023],
       [-0.00320918,  0.02041446],
       [ 0.01870406,  0.00493997],
       [-0.00325145, -0.01310843]])

## Train the model

In [3]:
trainset = np.load("dataset/human36M/processed_data/trainset.npy")
testset = np.load("dataset/human36M/processed_data/testset.npy")

In [4]:
from src.models.modelFactory import ModelFactory

In [5]:
model3d = ModelFactory.buildPose3DInterface()

INFO:tensorflow:Restoring parameters from parameters/pose3d/v1/checkpoint-113577


In [65]:
class BatchProvider:
    
    def __init__(self, features, values, batchSize, gaussianNoise = None):
        
        off = features.shape[0]%batchSize
        
        features = features[:-off,:]
        values = values[:-off,:]
        
        self.features = features
        self.values = values
        self.batchSize = batchSize
        self.gaussianNoise = gaussianNoise
        
    def shuffle(self):
        
        rndId = [i for i in range(self.features.shape[0])]
        random.shuffle(rndId)          
        self.features = self.features[rndId, :]
        self.values = self.values[rndId, :]
        
    def size(self):
        return int(self.features.shape[0]/self.batchSize)
    
    def getBatch(self, batchId):
        features = self.features[(batchId*self.batchSize):((batchId+1)*self.batchSize),:]
        features = features + self.gaussianNoise.getBatch(self.batchSize) if self.gaussianNoise != None else features
        values = self.values[(batchId*self.batchSize):((batchId+1)*self.batchSize),:]
        return (features,values)
        

In [7]:
train_features = np.hstack([trainset[:,:,0], trainset[:,:,2]])
train_values = trainset[:,:,1]

test_features = np.hstack([testset[:,:,0], testset[:,:,2]])
test_values = testset[:,:,1]

In [201]:
gNoise = GaussianNoise([0.028/2 for _ in range(13*2)])

In [202]:
bProv = BatchProvider(train_features, train_values, 16, gNoise)

In [214]:
feat,val = bProv.getBatch(0)

visualize_prediction(feat[0],val[0],val[0])

// TODO : try feature normalization

In [215]:
def train(session, model, trainFeat, trainValues, testFeat, testValues, totalEpochs, dropout, saveDir, trainGaussianNoise=None, loadSavedModel=-1):
    
    log_every_n_batches = 50
    test_every_n_batches = 10000
    
    batchTrainPro = BatchProvider(trainFeat, trainValues, model.batch_size, trainGaussianNoise)
    
    batchTestPro = BatchProvider(testFeat, testValues, model.batch_size)
    
    current_step = 0 if loadSavedModel <= 0 else loadSavedModel + 1
    print(current_step)
    
    for epoch in range(totalEpochs):

        loss = 0.
        
        for batchId in range(batchTrainPro.size()):
            
            feat, values = batchTrainPro.getBatch(batchId)
            step_loss, loss_summary, lr_summary, _ = model.step(session, feat, values, dropout)
            
            if (batchId+1) % log_every_n_batches == 0:
                # Log and print progress every log_every_n_batches batches
                model.train_writer.add_summary( loss_summary, current_step )
                model.train_writer.add_summary( lr_summary, current_step )    
                print( str(loss/(batchId+1)) +"     =at=> " +str((batchId+1)) +"/" +str((batchTrainPro.size())))
                
                
            if (batchId+1)% test_every_n_batches == 0: 
                testLoss = 0
                for batchTestId in range(batchTestPro.size()):
                    feat, values = batchTestPro.getBatch(batchTestId)
                    test_step_loss, _,  _ = model.step(session, feat, values, 1.0, False)
                    testLoss += test_step_loss
                print("================================= TEST  : "+str(testLoss/batchTestPro.size()))
                
                    
            loss += step_loss
            current_step += 1
            
        model.saver.save(session, os.path.join(saveDir, 'checkpoint'), global_step=current_step )

In [217]:
train(session, model, train_features, train_values, test_features, test_values, 10, 0.7, "dataset/human36M/processed_data/weights", gNoise, currModelLoadedStep)

49213
0.0061833903566002844     =at=> 50/32182
0.006976122288033366     =at=> 100/32182
0.010149829977502425     =at=> 150/32182
0.013239217237569392     =at=> 200/32182
0.01337860163860023     =at=> 250/32182
0.012726634923989575     =at=> 300/32182
0.012516843443736434     =at=> 350/32182
0.01251583237259183     =at=> 400/32182
0.011906484676421516     =at=> 450/32182
0.012057123446371406     =at=> 500/32182
0.011791637643579055     =at=> 550/32182
0.016148191284155473     =at=> 600/32182
0.01631516095573226     =at=> 650/32182
0.016478680292743124     =at=> 700/32182
0.016213297148856023     =at=> 750/32182
0.015720543085772077     =at=> 800/32182
0.015311337011208867     =at=> 850/32182
0.014904904483248375     =at=> 900/32182
0.01446042635012418     =at=> 950/32182
0.01414120240160264     =at=> 1000/32182
0.01382799332052292     =at=> 1050/32182
0.01383894958046519     =at=> 1100/32182
0.01367649519674318     =at=> 1150/32182
0.013468979058088734     =at=> 1200/32182
0.01327351654

0.013983724349921551     =at=> 9900/32182
0.014035789364078387     =at=> 9950/32182
0.01403236506744288     =at=> 10000/32182
================================= TEST  : 0.035373401076669614
0.014009707127049083     =at=> 10050/32182
0.013988532983896202     =at=> 10100/32182
0.013963618312269596     =at=> 10150/32182
0.013957783127746854     =at=> 10200/32182
0.013942202835252918     =at=> 10250/32182
0.013923524832466021     =at=> 10300/32182
0.013897012667595476     =at=> 10350/32182
0.013863929893973713     =at=> 10400/32182
0.013840111763412623     =at=> 10450/32182
0.013850004306617415     =at=> 10500/32182
0.013893704685665031     =at=> 10550/32182
0.014320599765556354     =at=> 10600/32182
0.014806536973733892     =at=> 10650/32182
0.014975570159698793     =at=> 10700/32182
0.015289604838092833     =at=> 10750/32182
0.015675388285825546     =at=> 10800/32182
0.016005050476387245     =at=> 10850/32182
0.016277729408460916     =at=> 10900/32182
0.016580018947373005     =at=> 10950/

0.01603182557254436     =at=> 19450/32182
0.01600515572231215     =at=> 19500/32182
0.01600660435251577     =at=> 19550/32182
0.016024186846845288     =at=> 19600/32182
0.016025502013727436     =at=> 19650/32182
0.016018092754346597     =at=> 19700/32182
0.01603286430403543     =at=> 19750/32182
0.01623133947591843     =at=> 19800/32182
0.016538882869699272     =at=> 19850/32182
0.016714555763033448     =at=> 19900/32182
0.016745841055816706     =at=> 19950/32182
0.016740331288811285     =at=> 20000/32182
================================= TEST  : 0.041151955806532615
0.016734939910009263     =at=> 20050/32182
0.016729961806683527     =at=> 20100/32182
0.01672224069461365     =at=> 20150/32182
0.01670038769120756     =at=> 20200/32182
0.016673279977760014     =at=> 20250/32182
0.016653551237871     =at=> 20300/32182
0.01672527484686162     =at=> 20350/32182
0.01672280657119985     =at=> 20400/32182
0.016708304521691482     =at=> 20450/32182
0.016725456939748966     =at=> 20500/32182
0.0

0.01954440503154785     =at=> 29000/32182
0.019528767916248878     =at=> 29050/32182
0.019511624551866256     =at=> 29100/32182
0.019489453952127707     =at=> 29150/32182
0.019473636908610575     =at=> 29200/32182
0.019452558033602137     =at=> 29250/32182
0.019431320386934778     =at=> 29300/32182
0.019408725152895635     =at=> 29350/32182
0.019389334479437945     =at=> 29400/32182
0.019382987538196863     =at=> 29450/32182
0.019370512507015335     =at=> 29500/32182
0.019351190595675652     =at=> 29550/32182
0.019330491788966134     =at=> 29600/32182
0.019312473039349312     =at=> 29650/32182
0.0193148338586219     =at=> 29700/32182
0.01930711869413753     =at=> 29750/32182
0.0193229915990827     =at=> 29800/32182
0.019321127721585077     =at=> 29850/32182
0.01934259484385383     =at=> 29900/32182
0.01935115475019048     =at=> 29950/32182
0.019336298283480574     =at=> 30000/32182
================================= TEST  : 0.020549072164413297
0.019321939239829326     =at=> 30050/32182

0.015426610042341053     =at=> 6600/32182
0.015375311434535044     =at=> 6650/32182
0.015433739604664716     =at=> 6700/32182
0.015410498018238555     =at=> 6750/32182
0.015366675287574147     =at=> 6800/32182
0.0155173041998837     =at=> 6850/32182
0.01551895335185058     =at=> 6900/32182
0.01553958266501134     =at=> 6950/32182
0.015522338485844167     =at=> 7000/32182
0.01547331757295269     =at=> 7050/32182
0.015419540173426585     =at=> 7100/32182
0.015466849862539269     =at=> 7150/32182
0.015532076831829424     =at=> 7200/32182
0.015524199940713829     =at=> 7250/32182
0.01551034775179849     =at=> 7300/32182
0.01546061398353757     =at=> 7350/32182
0.0154233718954807     =at=> 7400/32182
0.015381456263325858     =at=> 7450/32182
0.015332440886646509     =at=> 7500/32182
0.015335536523697442     =at=> 7550/32182
0.015285823630493772     =at=> 7600/32182
0.015244976870892143     =at=> 7650/32182
0.015197758318027844     =at=> 7700/32182
0.015249306541747384     =at=> 7750/32182
0

0.016724770694791984     =at=> 16250/32182
0.016709024669006867     =at=> 16300/32182
0.01669319504264611     =at=> 16350/32182
0.01666798794023812     =at=> 16400/32182
0.01663839234581987     =at=> 16450/32182
0.016643876627193424     =at=> 16500/32182
0.01664641079154524     =at=> 16550/32182
0.016741810548561064     =at=> 16600/32182
0.016750209100898243     =at=> 16650/32182
0.016731403480056663     =at=> 16700/32182
0.01671113201918136     =at=> 16750/32182
0.016688136984046736     =at=> 16800/32182
0.016664753109762495     =at=> 16850/32182
0.016641669240249762     =at=> 16900/32182
0.016608771402780297     =at=> 16950/32182
0.01658318247801542     =at=> 17000/32182
0.016582827188212513     =at=> 17050/32182
0.016574581178941367     =at=> 17100/32182
0.01659141440925381     =at=> 17150/32182
0.016688295182708358     =at=> 17200/32182
0.016702875840686856     =at=> 17250/32182
0.016711554502176033     =at=> 17300/32182
0.016724618254155617     =at=> 17350/32182
0.0167187721507514

0.01889551302750212     =at=> 25800/32182
0.01888258299323761     =at=> 25850/32182
0.018888053459485698     =at=> 25900/32182
0.018878492232491054     =at=> 25950/32182
0.018864890626166017     =at=> 26000/32182
0.018844933760622355     =at=> 26050/32182
0.018830741786440813     =at=> 26100/32182
0.018813323409690115     =at=> 26150/32182
0.01879289015389993     =at=> 26200/32182
0.01877413031316939     =at=> 26250/32182
0.018751203111946554     =at=> 26300/32182
0.018735324715138668     =at=> 26350/32182
0.018723280095595825     =at=> 26400/32182
0.018721564237762504     =at=> 26450/32182
0.018726092740357414     =at=> 26500/32182
0.018744784220132326     =at=> 26550/32182
0.018770465472375247     =at=> 26600/32182
0.01892511328796753     =at=> 26650/32182
0.019012954012213507     =at=> 26700/32182
0.01911288416040104     =at=> 26750/32182
0.019185095278210867     =at=> 26800/32182
0.019343814686161     =at=> 26850/32182
0.0194402847427539     =at=> 26900/32182
0.01965362905352748   

0.01864037903900625     =at=> 3300/32182
0.018547881761593606     =at=> 3350/32182
0.018710735240841613     =at=> 3400/32182
0.01852869036965126     =at=> 3450/32182
0.01837477977647047     =at=> 3500/32182
0.01819976113612016     =at=> 3550/32182
0.018031861194888026     =at=> 3600/32182
0.01789385955161989     =at=> 3650/32182
0.017715017146091104     =at=> 3700/32182
0.01756376358581086     =at=> 3750/32182
0.01742305395432356     =at=> 3800/32182
0.017455627122083556     =at=> 3850/32182
0.01741265466096453     =at=> 3900/32182
0.01743542542722359     =at=> 3950/32182
0.017325651729188395     =at=> 4000/32182
0.01725168004833208     =at=> 4050/32182
0.01723356953498385     =at=> 4100/32182
0.01716088594638754     =at=> 4150/32182
0.01702278141137434     =at=> 4200/32182
0.01688649879845188     =at=> 4250/32182
0.01675356145536657     =at=> 4300/32182
0.01663802143951728     =at=> 4350/32182
0.016518008186619474     =at=> 4400/32182
0.016395131611223385     =at=> 4450/32182
0.016286

KeyboardInterrupt: 

In [11]:
def predict(session, model, inputRecord):
    
    res = session.run(model.outputs, feed_dict={model.encoder_inputs: inputRecord,
                                                       model.isTraining: False,
                                                       model.dropout_keep_prob: 1.0
                                                       })
    return res

## visualize results 

In [11]:
def visualize_prediction(features, gt_res, predicted_res):
  
    x = np.expand_dims(features[:13],axis=1)
    y_gt = np.expand_dims(gt_res,axis=1)
    y_pred = np.expand_dims(predicted_res,axis=1)
    z = np.expand_dims(features[13:],axis=1)
    
    gt_pos = np.hstack([x,y_gt,z])
    predicted_pos = np.hstack([x,y_pred,z])
    
    h36m.visualize_gt_vs_pred(gt_pos, predicted_pos)
    

In [16]:
entryId = 6100

pred = model3d.predict(np.expand_dims(test_features[entryId], axis=0))[0]

visualize_prediction(test_features[entryId], test_values[entryId], pred)



## RNN TEST

In [3]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout


class Model:

    def __init__(self, totalHistoryFrames, featuresByFrame, outputSize):

        totalUnits = int(0.75*totalHistoryFrames) + 1

        self.regressor = Sequential()

        self.regressor.add(LSTM(units=totalUnits, input_shape=(totalHistoryFrames, featuresByFrame), return_sequences=True))
        self.regressor.add(Dropout(0.2))

        self.regressor.add(LSTM(units=totalUnits, return_sequences=False))
        self.regressor.add(Dropout(0.2))

        self.regressor.add(Dense(units=outputSize))

        self.regressor.compile(optimizer='adam', loss='mean_squared_error')


    """
    x : <total_examples, totalHistoryFrames, featuresByFrame>
    y : <total_examples, outputSize>
    """
    def train(self, x, y):
        self.regressor.fit(x,y)


    def predict(self, x):
        return self.predict(x)



Using TensorFlow backend.
